# SEIRX model runs for test technology and screening frequency investigation

In [11]:
import networkx as nx
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
#import os
#from os.path import expanduser
#home = expanduser("~")
#os.chdir(home+'/Projects/COVID_agent_based_simulation/small_community_SEIRX/src/')
#os.getcwd()
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
from scseirx import analysis_functions as af # custom analysis functions
import data_creation_functions_vaccination as dcf

from importlib import reload

## Parameters

### Simulation parameters and measures in place

In [12]:
with open('params/screening_frequency_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/screening_frequency_measures.json') as json_file:
    measures = json.load(json_file)


### Screening parameters

In [13]:
e_vaccination_probabilities = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
r_vaccination_probabilities = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

screening_params = [(i, j, k)\
              for i in index_cases \
              for j in e_vaccination_probabilities \
              for k in r_vaccination_probabilities]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 242 parameter combinations


In [14]:
def run(param_list, N_runs=10):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # paths for data I/O
    contact_network_src = '../data/contact_networks'

    # extract the simulation parameters from the parameter list
    index_case, e_vaccination_probability, r_vaccination_probability = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, index_case, 
                e_vaccination_probability, r_vaccination_probability, measures,
                simulation_params, contact_network_src)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case,
                        e_vaccination_probability, r_vaccination_probability)

    return row

In [15]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_probability', 'employee_vaccination_probability']
              #'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

#results.to_csv(join(dst,'simulations_test_technology_screening_interval.csv'),
               #index=False)
results.head(3)

100%|██████████| 242/242 [00:42<00:00,  5.76it/s]


CPU times: user 3.2 s, sys: 221 ms, total: 3.42 s
Wall time: 44.8 s


,index_case,resident_vaccination_probability,employee_vaccination_probability,N_diagnostic_tests_0.025,N_diagnostic_tests_0.10,N_diagnostic_tests_0.25,N_diagnostic_tests_0.75,N_diagnostic_tests_0.90,N_diagnostic_tests_0.975,N_diagnostic_tests_mean,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,employee,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.392202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,employee,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.581571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,employee,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.103872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
dst = '../data/simulation_results'

In [17]:
results.to_csv(join(dst,'simulations_vaccination_rate.csv'),
               index=False)